# Setup

## Loading data

In [1]:
# Libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import altair as alt
import os.path
import getpass
import platform
from pandasql import sqldf

#Data
username = getpass.getuser()

print("Username: ", username)
pathPart1 = "/Users/"
pathPart2 = "/Github/SCGProject/Datasets/"
complePath = pathPart1+username+pathPart2

if platform.system() == "Darwin":
    if(username == "marcovinciguerra"):
        dfCambio = pd.read_csv(
            complePath+"/CorrectedDatasets/dfCambio.csv")

        dfClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dfClienti.csv")

        dfConsumi = pd.read_csv(
             complePath + "/CorrectedDatasets/dfConsumi.csv")

        dfCostoOrarioConsuntivo = pd.read_csv(
            complePath+"/CorrectedDatasets/dfConsuntivo.csv")

        dfCostoOrarioBudget = pd.read_csv(
            complePath+"CorrectedDatasets/dfCostoOrario.csv")

        dfImpiegoRisorse = pd.read_csv(
         complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

        dftuttiClienti = pd.read_csv(
         complePath+"/CorrectedDatasets/dftuttiClienti.csv")

        dfVendite = pd.read_csv(
         complePath+"/CorrectedDatasets/dfVendite.csv")

    elif(username == "davidguzman"):
            pathPart1 = "/Users/"
            pathPart2 = "/documents/Github/SCGProject/Datasets/"
            complePath = pathPart1+username+pathPart2

            dfCambio = pd.read_csv(
                complePath+"/CorrectedDatasets/dfCambio.csv")

            dfClienti = pd.read_csv(
                complePath+"/CorrectedDatasets/dfClienti.csv")

            dfConsumi = pd.read_csv(
                complePath + "/CorrectedDatasets/dfConsumi.csv")

            dfCostoOrarioConsuntivo = pd.read_csv(
                complePath+"/CorrectedDatasets/dfConsuntivo.csv")

            dfCostoOrarioBudget = pd.read_csv(
                complePath+"CorrectedDatasets/dfCostoOrario.csv")

            dfImpiegoRisorse = pd.read_csv(
            complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

            dftuttiClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dftuttiClienti.csv")

            dfVendite = pd.read_csv(
            complePath+"/CorrectedDatasets/dfVendite.csv")

elif platform.system() == "Linux":
    complePath = "/home/alinux/unibg/git_repo/SCGProject"
    pathPart1 = "/Users/"
    pathPart2 = "/documents/Github/SCGProject/Datasets/"
    complePath = pathPart1+username+pathPart2
    complePath = '/home/alinux/unibg/git_repo/SCGProject/Datasets/'

    dfCambio = pd.read_csv(
        complePath+"/CorrectedDatasets/dfCambio.csv")

    dfClienti = pd.read_csv(
        complePath+"/CorrectedDatasets/dfClienti.csv")

    dfConsumi = pd.read_csv(
        complePath + "/CorrectedDatasets/dfConsumi.csv")

    dfCostoOrarioConsuntivo = pd.read_csv(
        complePath+"/CorrectedDatasets/dfConsuntivo.csv")

    dfCostoOrarioBudget = pd.read_csv(
        complePath+"CorrectedDatasets/dfCostoOrario.csv")

    dfImpiegoRisorse = pd.read_csv(
        complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

    dftuttiClienti = pd.read_csv(
        complePath+"/CorrectedDatasets/dftuttiClienti.csv")

    dfVendite = pd.read_csv(
        complePath+"/CorrectedDatasets/dfVendite.csv")

dfCambio["TassoCambioMedio"] = dfCambio["TassoCambioMedio"].str.replace(',','.')

Username:  marcovinciguerra


## Creating the dataset with no intermediate phases

### Budget

In [12]:
dfConsumi.rename(columns={'Budget/cons': 'status'}, inplace=True)
qtaArtBudget = sqldf(
    'select NrArticolo, count(*) as qta from dfConsumi where status = "BUDGET" group by NrArticolo')

dfVendite.rename(columns={'budget/cons': 'status'}, inplace=True)
qtaArtVendBudget = sqldf(
    'select NrArticolo, count(*) as qta from dfVendite where status = "BUDGET" group by NrArticolo')

artProdeVendBudget = sqldf(
    'select P.NrArticolo from qtaArtBudget P join qtaArtVendBudget V on P.NrArticolo = V.NrArticolo')


consumiB = sqldf('select * from dfConsumi where status = "BUDGET"')


# ho trovato articoli che hanno consumato risorse a budget ma non sono stati messi tra le vendite a budget
artProdeNONVendBudget = sqldf(
    'select * from consumiB group by NrArticolo having NrArticolo not in (select NrArticolo from artProdeVendBudget)')

# prendo gli articoli che usano come materia prima almeno un articolo prodotto internamente
# sqldf('select NrMovimento, status, C.NrArticolo, CodiceMP, NrDocumento, QuantityMPImpiegata, ImportoCostoTOTALE from consumiB C join artProdeNONVendBudget A on C.CodiceMP = A.NrArticolo')

costoPerArticoloBudget = sqldf(
    'select NrArticolo, sum(ImportoCostoTOTALE) as costoMpPerArt from consumiB where NrArticolo not in (select NrArticolo from artProdeNONVendBudget) group by NrArticolo')
sqldf('select sum(costoMpPerArt) as costoTotBudget from costoPerArticoloBudget')

mpB = sqldf('''select NrMovimento,status,CodiceMP,NrArticolo, NrDocumento, sum(QuantityMPImpiegata), sum(ImportoCostoTOTALE) from consumiB group by NrArticolo''')
prodInternaB = sqldf('''select A.NrMovimento, A.status, A.CodiceMP, A.NrArticolo, A.NrDocumento, A.QuantityMPImpiegata, A.ImportoCostoTOTALE
         from consumiB A join mpB B on A.CodiceMP = B.NrArticolo''')

mpIntermediB = sqldf('''select distinct CodiceMP from prodInternaB''')
consumiBNOArtInterm = sqldf(
    '''select * from consumiB where NrArticolo not in (select CodiceMP from mpIntermediB)''')


### Consuntivo

In [16]:
consumiC = sqldf(
    '''select * from dfConsumi where status = "CONSUNTIVO" or status = "Consuntivo"''')
mpC = sqldf('''select NrMovimento,status,CodiceMP,NrArticolo, NrDocumento, sum(QuantityMPImpiegata), sum(ImportoCostoTOTALE) from consumiC group by NrArticolo''')
#A.NrArticolo, A.CodiceMP, A.NrDocumento, A.QuantityMPImpiegata, A.ImportoCostoTOTALE, A.CodiceMP
prodInternaC = sqldf('''select A.NrMovimento, A.status, A.CodiceMP, A.NrArticolo, A.NrDocumento, A.QuantityMPImpiegata, A.ImportoCostoTOTALE
         from consumiC A join mpC B on A.CodiceMP = B.NrArticolo''')
mpIntermediC = sqldf('''select distinct CodiceMP from prodInternaC''')
consumiCNOArtInterm = sqldf(
    '''select * from consumiC where NrArticolo NOT IN (select CodiceMP from mpIntermediC)''')


### UNIONE DELLE DUE TABELLE PER FARE IL NUOVO DATASET

In [18]:
dfConsumi = sqldf('''  SELECT *
                            FROM consumiBNOArtInterm
                            UNION
                            SELECT *
                            FROM consumiCNOArtInterm
                            ''')
del dfConsumi['Unnamed: 0']
dfConsumi


,NrMovimento,status,CodiceMP,NrArticolo,NrDocumento,QuantityMPImpiegata,ImportoCostoTOTALE
0,28854,BUDGET,ART0001074,ART0000018,ODP17-05342,30.0,5.15
1,66819,CONSUNTIVO,ART0001074,ART0000018,ODP19-05998,30.0,5.15
2,28853,BUDGET,ART0005696,ART0000041,ODP17-05342,4.0,92.00
3,66818,CONSUNTIVO,ART0005696,ART0000041,ODP19-05998,4.0,88.00
4,28855,BUDGET,ART0005697,ART0000042,ODP17-05342,4.0,104.00
...,...,...,...,...,...,...,...
4061,66172,CONSUNTIVO,ART0005850,ART0005136,ODP19-02057,155.0,1.23
4062,66235,CONSUNTIVO,ART0006245,ART0005136,ODP19-02528,700.0,0.00
4063,66446,CONSUNTIVO,ART0006381,ART0005136,ODP19-02674,300.0,1.62
4064,66594,CONSUNTIVO,ART0006345,ART0005136,ODP19-06688,9.0,0.00
